<a href="https://colab.research.google.com/github/ssook1222/Data-Analysis-Crush/blob/main/1%ED%9A%8C%EB%8F%85/1%EC%98%81%EC%97%AD/task1_103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 서울시 따릉이 이용정보 데이터

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


### Question 31

대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를  출력하라

In [10]:
df.groupby('대여일자', as_index = False).size().sort_values('size', ascending = False).head(1).values[0][0]

'2021-06-04'

### Question 32

각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로  출력하라

In [15]:
import datetime as dt

df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df.대여일자.dt.day_name()

df.groupby('day_name').size()

day_name
Friday       108877
Monday       107568
Saturday     101299
Sunday        90918
Thursday      55977
Tuesday      103704
Wednesday    110607
dtype: int64

### Question 33

각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라

In [29]:
df[['day_name', '대여소번호']].groupby(['day_name', '대여소번호'], as_index = False).size().sort_values('size', ascending = False).drop_duplicates('day_name', keep = 'first')

,day_name,대여소번호,size
5191,Saturday,502,378
7611,Sunday,502,372
14820,Wednesday,502,282
348,Friday,502,277
12421,Tuesday,502,267
2769,Monday,502,242
11482,Thursday,2715,137


### Question 34

나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라.  
일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

In [30]:
df.columns

Index(['대여일자', '대여시간', '대여소번호', '대여구분코드', '성별', '연령대코드', '이용건수', '운동량', '탄소량',
       '이동거리', '사용시간', 'day_name'],
      dtype='object')

In [46]:
df34 = df[(df.대여구분코드 == '일일권')]
df34_1 = df34.groupby('연령대코드', as_index = False).size()
df34_2 = df.groupby('연령대코드', as_index = False).size()

df34_total = df34_1.merge(df34_2, on='연령대코드', how='inner')
df34_total['rate'] = df34_total['size_x'] / df34_total['size_y']
display(df34_total.sort_values('연령대코드'))

,연령대코드,size_x,size_y,rate
0,20대,59389,247561,0.239896
1,30대,31124,186722,0.166686
2,40대,12692,114799,0.110558
3,50대,4606,70428,0.065400
4,60대,819,19288,0.042462
5,70대~,158,3227,0.048962
6,~10대,11540,36925,0.312525


### Question 35

연령대별 평균 이동거리를 구하여라

In [49]:
df.groupby(['연령대코드']).이동거리.mean()

연령대코드
20대     3211.890552
30대     3341.443859
40대     3514.857416
50대     3593.668100
60대     3538.145737
70대~    3085.039641
~10대    2634.426279
Name: 이동거리, dtype: float64

### Question 36

연령대 코드가 20대인 데이터를 추출하고, 이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.    
최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라

In [58]:
filter = df[df['연령대코드'] == '20대'].이동거리.mean()

df36 = df[(df.이동거리 >= filter)].sort_values(['대여일자', '대여소번호'], ascending = False).head(200)
print(round(df36.탄소량.astype('float').mean(), 3))

1.513


### Question 37

6월 7일 ~10대의 “이용건수”의 중앙값은?

In [62]:
df[(df['연령대코드'] == '~10대') & (df['대여일자'] == pd.to_datetime('2021-06-07'))].이용건수.median()

1.0

### Question 38

평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를  출력하라

In [83]:
df38 = df[(df.대여시간.isin([6,7,8]) & (df.day_name.isin(['Tuesday', 'Wednesday', 'Thursday', 'Friday','Monday'])))]
df38_total = df38.groupby(['대여시간','대여소번호']).size().to_frame('이용 횟수')
df38_total.sort_values(['대여시간', '이용 횟수'], ascending = False).groupby('대여시간').head(3)

이용 횟수
대여시간 대여소번호       
8    2701     119
     646      115
     1152      92
7    259      104
     230       77
     726       77
6    2744      45
     1125      40
     1028      36

### Question 39

이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라

In [87]:
df[(df.이동거리.mean() < df.이동거리)].이동거리.std()

5092.139707505356

### Question 40

남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라

In [95]:
df_male = df[(df['성별'] == 'M') | (df['성별'] == 'm')]
df_female = df[(df['성별'] == 'F') | (df['성별'] == 'f')]

male_mean = df_male.이동거리.mean()
female_mean = df_female.이동거리.mean()


ans_dict = {'sex':['남', '여'], '평균 이동거리':[male_mean, female_mean]}
ans = pd.DataFrame(ans_dict)

display(ans)

,sex,평균 이동거리
0,남,3209.110871
1,여,3468.575025
